In [167]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
import pickle

#filename queue
filename_queue=glob.glob('./rawdata/data[1-5].csv')
#Create return path
return_path='./rawdata/data.csv'
create_file=open(return_path,'w')

In [168]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
index=[]
label_count=np.zeros(16)
traininglabel_count=np.zeros(16)
testlabel_count=np.zeros(16)




#return_file=open(return_path,'a')
#writer=csv.writer(return_file)
#writer.writerow()
n_data=[]

for filename in filename_queue:
    file=open(filename, newline='')
    
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        labl=int(row[3])
        if labl not in index:
            label_count[labl]+=1
            for x in range(4):
                X_acc=list(row[4*i+4+x] for i in range(150))
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Z_gyro=list(row[4*i+3004+x] for i in range(150))

                window=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                    [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])

                window=np.array([[X_acc[0:25],Y_acc[0:25],Z_acc[0:25],X_gyro[0:25],Y_gyro[0:25],Z_gyro[0:25]],
                                  [X_acc[25:50],Y_acc[25:50],Z_acc[25:50],X_gyro[25:50],Y_gyro[25:50],Z_gyro[25:50]],
                                  [X_acc[50:75],Y_acc[50:75],Z_acc[50:75],X_gyro[50:75],Y_gyro[50:75],Z_gyro[50:75]],
                                  [X_acc[75:100],Y_acc[75:100],Z_acc[75:100],X_gyro[75:100],Y_gyro[75:100],Z_gyro[75:100]],
                                  [X_acc[100:125],Y_acc[100:125],Z_acc[100:125],X_gyro[100:125],Y_gyro[100:125],Z_gyro[100:125]],
                                  [X_acc[125:150],Y_acc[125:150],Z_acc[125:150],X_gyro[125:150],Y_gyro[125:150],Z_gyro[125:150]]])

                #5 features  what else?2
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)

                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)

                if (label_count[labl]%5!=1):
                    training_set.append(window_feature)
                    training_label.append(labl)
                    traininglabel_count[labl]+=1

                else:
                    test_set.append(window_feature)
                    test_label.append(labl)
                    testlabel_count[labl]+=1
        

In [169]:
#print(window_feature.shape)
print(len(training_label))
print(len(test_label))

1528
408


In [170]:
#SVM
#prepare the data
X=training_set
y=training_label
#prepare the model
SVC = sklearn.svm.SVC(gamma='scale',tol=0.1,probability=True)

SVC.fit(X,y)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

test_pred1=SVC.predict(test_set)
test_pred_proba=SVC.predict_proba(test_set)

ohc=OneHotEncoder(categories=[range(16)])
onehot_pred1=test_pred1.reshape(-1,1)
onehot_pred1=ohc.fit_transform(onehot_pred1).toarray()
#print(test_pred1)
#print(onehot_pred1)

print(accuracy_score(test_pred1,test_label))
#print(SVC.n_support_)
#confusion_matrix(test_pred1, test_label)

0.7009803921568627


In [171]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
KNN = NearestCentroid()
KNN.fit(X, y)
test_pred2=KNN.predict(test_set)
print(accuracy_score(test_pred2,test_label))

onehot_pred2=test_pred2.reshape(-1,1)
onehot_pred2=ohc.fit_transform(onehot_pred2).toarray()
#print(onehot_pred2)
#confusion_matrix(test_pred2, test_label)

0.6004901960784313


In [172]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
RF = RandomForestClassifier(200)
RF.fit(X, y)
test_pred_proba=RF.predict_proba(test_set)
print(np.shape(test_pred_proba))
#print(np.argmax(x) for x in test_pred_proba)
test_pred3=RF.predict(test_set)
#for x in test_pred_proba:
#    print(x)
print(accuracy_score(test_pred3,test_label))
#print(confusion_matrix(test_pred3, test_label))
onehot_pred3=test_pred2.reshape(-1,1)
onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
#print(onehot_pred3)

(408, 16)
0.8112745098039216


In [173]:
print(test_pred3)
np.shape(test_pred3)
newpred2=np.zeros(len(test_pred3))
print(np.shape(newpred2))

[ 7  7  7  7  9  9  9  9 10 10 10 10 10 10 10 10  9  9  9  9 10 10 10 10
 10 10 10 10 10 10 10 10  9  9  9  9  6  6  6  6  7  7  7  7  1  1  1  1
  6  6  6  6 14 14 14 14  0  0  0  0  0  0  0  0  6  6  6  6  5  5  5  5
  7  7  7  7  3  3  3  3 12 12 12 12  6  6  6  6 12 12 12 12  1  1  1  1
 12 10 12 12 12 12 12 12 12 12 12 12  6  6  6  6 12  9  9 12 12 12 12 12
  6  6  6  6  5  5  5  5  0  0  0  0 12 12 12 12 12 12 12 12  6  6  6  6
 12 12 12 12  0  0  0  0  5  5  5  5  0  0  0  0  3  3  3  3 12 12 12 12
  6  6  6  6  1  1  1  1  0  0  0  0  1  1  1  1 10 10 10 10 12 12 12 12
 13 13 13 13  8  8  8  8 14 14 14 14  8  8  8  8 14 14 14 14 13 13 13 13
  0  0  0  0 12 12 12 12  5  5  5  5  6  6  6  6  6  6  6  6 12 12 12 12
  1  1  1  1  5  5  5  5  6  6  6  6  3  3  3  3  7  7  7  7  8  8  8  8
 12 12 12 12  1  1  1  1  8  8  8  8  1  1  1  1  8  8  8  8  0  0  0  0
  4  4  4  4  8  8  8  8  4  4  4  4  7  7  7  7  4  4  4  4 10 10  3  3
  0  0  0  0  0  0  0  0  0  0  0  0  7  7  7  7  3

In [174]:
print(test_label)

[12, 12, 12, 12, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 12, 12, 12, 12, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 6, 6, 6, 6, 7, 7, 7, 7, 1, 1, 1, 1, 6, 6, 6, 6, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 5, 5, 5, 5, 7, 7, 7, 7, 3, 3, 3, 3, 4, 4, 4, 4, 6, 6, 6, 6, 12, 12, 12, 12, 1, 1, 1, 1, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 0, 0, 0, 0, 9, 9, 9, 9, 12, 12, 12, 12, 6, 6, 6, 6, 5, 5, 5, 5, 0, 0, 0, 0, 12, 12, 12, 12, 12, 12, 12, 12, 6, 6, 6, 6, 7, 7, 7, 7, 2, 2, 2, 2, 5, 5, 5, 5, 0, 0, 0, 0, 3, 3, 3, 3, 12, 12, 12, 12, 6, 6, 6, 6, 1, 1, 1, 1, 5, 5, 5, 5, 1, 1, 1, 1, 10, 10, 10, 10, 13, 13, 13, 13, 13, 13, 13, 13, 8, 8, 8, 8, 14, 14, 14, 14, 8, 8, 8, 8, 14, 14, 14, 14, 13, 13, 13, 13, 0, 0, 0, 0, 12, 12, 12, 12, 5, 5, 5, 5, 8, 8, 8, 8, 6, 6, 6, 6, 12, 12, 12, 12, 1, 1, 1, 1, 5, 5, 5, 5, 8, 8, 8, 8, 3, 3, 3, 3, 7, 7, 7, 7, 5, 5, 5, 5, 12, 12, 12, 12, 11, 11, 11, 11, 6, 6, 6, 6, 1, 1, 1, 1, 8, 8, 8, 8, 12, 12, 12, 12, 5, 5, 5, 5, 8, 8, 8, 8, 15, 15, 15,

In [175]:
array=np.array(test_pred_proba)
newpred2=np.zeros(len(test_pred3))
for x in range(len(test_pred_proba)):
    if(np.max(test_pred_proba[x])>0.3):
        newpred2[x]=test_pred3[x]
    else:
        newpred2[x]=-1
        i=0
        #print(np.shape(np.argmax(test_pred_proba[x])))
print(np.shape(newpred2))

(408,)


In [176]:
print(accuracy_score(newpred2,test_label))

0.7549019607843137


In [177]:
#ensemble 1 : unanimous consensus
ensemble1_pred= onehot_pred1*onehot_pred2*onehot_pred3
#print(ensemble1_pred)
ensemble1_pred=np.argmax(ensemble1_pred,axis=1)
#print(ensemble1_pred)

In [178]:
#ensemble 2 : majority consensus
ensemble2_pred= (onehot_pred1+onehot_pred2+onehot_pred3)/2
ensemble2_pred= ensemble2_pred.astype('int64') 
ensemble2_pred= ensemble2_pred.astype('float64') 
ensemble2_pred=np.argmax(ensemble2_pred,axis=1)
#print(ensemble2_pred)
#print(test_label)
print(accuracy_score(ensemble2_pred,test_label))


0.6004901960784313


In [179]:
#save the model

from joblib import dump, load
filename='./svm.sav'
pickle.dump(SVC,open(filename,'wb'))

filename='./knn.sav'
pickle.dump(KNN,open(filename,'wb'))
dump(KNN,'knn.joblib')

filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')


['rf.joblib']

In [180]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

ETC=ExtraTreesClassifier(300)
DTC=DecisionTreeClassifier()

ETC.fit(X,y)
DTC.fit(X,y)
test_pred4=ETC.predict(test_set)
test_pred5=DTC.predict(test_set)
print(accuracy_score(test_pred4,test_label))
print(accuracy_score(test_pred5,test_label))



0.8112745098039216
0.6127450980392157


In [181]:
test_pred_proba2=ETC.predict_proba(test_set)
array=np.array(test_pred_proba2)
newpred3=np.zeros(len(test_pred4))
for x in range(len(test_pred_proba2)):
    if(np.max(test_pred_proba2[x])>0.2):
        newpred3[x]=test_pred4[x]
    else:
        newpred3[x]=-1
        i=0
        #print(np.shape(np.argmax(test_pred_proba[x])))
print(np.shape(newpred3))
print(newpred3)
print(accuracy_score(newpred3,test_label))

(408,)
[ 7.  7.  7.  7.  9.  9.  9.  9. 10. 10. 10. 10. 10. 10. 10. 10.  9.  9.
  9.  9. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.  9.  9.  9.  9.
  6.  6.  6.  6.  7.  7.  7.  7.  1.  1.  1.  1.  6.  6.  6.  6. 14. 14.
 14. 12.  0.  0.  0.  0.  0.  0.  0.  0.  6.  6.  6.  6.  5.  5.  5.  5.
  7.  7.  7.  7.  9.  9. 10.  3. -1. -1. -1. -1.  6.  6.  6.  6. 12. 12.
 12. 12.  1.  1.  1.  1. 10. 10. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.
  6. -1.  6. -1. 12. 12. 12. 12. 12. 12. 12. 12.  6.  6.  6.  6.  5.  5.
  5.  5.  0.  0.  0.  0. 12. 12. 12. 12. 12. 12. 12. 12.  6.  6.  6.  6.
  0.  0.  0.  0.  0.  0.  0.  0.  5.  5.  5.  5.  0.  0.  0.  0.  3.  3.
  3.  3. 12. 12. 12. 12.  6.  6.  6.  6.  1.  1.  1.  1.  0.  0.  0.  0.
  1.  1.  1.  1. 10. 10. 10. 10. 12. 12. 12. 12. 13. 13. 13. 13.  8.  8.
  8.  8. 14. 14. 14. 14.  8.  8.  8.  8. 14. 14. 14. 14. 13. 13. 13. 13.
  0.  0.  0.  0. 12. 12. 12. 12.  5.  5.  5.  5.  6.  6.  6.  6.  6.  6.
  6.  6. 12. 12. 12. 12.  1.  1.  1.  1.  5.

In [182]:
accuracy_per_label=[]

accuracy_per_label=confusion_matrix(test_pred4, test_label)/testlabel_count
accuracy_per_label.diagonal()
accuracy_per_label=confusion_matrix(test_pred5, test_label)/testlabel_count
accuracy_per_label.diagonal()

array([0.81818182, 0.75      , 0.5       , 0.6       , 0.33333333,
       0.5       , 0.525     , 0.66666667, 0.33333333, 0.67857143,
       0.91666667, 0.        , 0.66666667, 0.33333333, 0.        ,
       0.5       ])

In [183]:
index=[2,11,8]
